In [1]:
import os

In [2]:
!ls

ArthurMethodCode.ipynb
CascadeTwoPhase.PNG
DifferentError.PNG
Error.PNG
FinalCode.ipynb
FinalCode_Avni_Local1.ipynb
Loss.PNG
Progress.PNG
ResizedAndLessPatches.ipynb
Segmentation_data.zip
Test_seg.zip
ThePromisedLand.ipynb
final_presentation_example.mov
fuck.png
slice_126_4.jpg
slice_126_GT.jpg
trial_0001_input_cascasde_acc.h5
trial_MFCcascade_acc.h5


In [5]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import metrics

In [6]:
data_path = r'D:\Grad School Course Work\CS 6501 - MLIA\Test_seg' #Change this to wherever the Data Folder is located

In [7]:
def convert_y_2D_y_3D(img): #OHEs the labels for the entire image efficiently
    layer1 = 1*(img == 1)
    layer2 = 1*(img == 2)
    layer3 = 1*(img == 3)
    layer4 = 1*(img == 4)
    return np.dstack([layer1, layer2, layer3, layer4])

In [8]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Data Visualisations

In [11]:
def model_gen(input_dim,x,y,slice_no):
    X1 = []
    X2 = []
    Y = []
    print(x.shape)
    for i in range(int((input_dim)/2),x.shape[0]-int((input_dim)/2), input_dim%3):
        for j in range(int((input_dim)/2),x.shape[1]-int((input_dim)/2), input_dim%3):
          #Filtering all 0 patches
          if(np.count_nonzero(x[i-16:i+17,j-16:j+17,:]) > 100 or x[i,j] != 0): #Check if theres a meaningufl amount of data in the patch
#             print("({0},{1})".format(i,j))
            X2.append(x[i-16:i+17,j-16:j+17,:])
            X1.append(x[i-int((input_dim)/2):i+int((input_dim)/2)+1,j-int((input_dim)/2):j+int((input_dim)/2)+1,:])
#             print("y_i_j = ", y[i,slice_no,j])
            Y.append(y[slice_no,i,j]) #Was i, slice_no, j
      
      
    X1 = np.asarray(X1)
    X1 = X1[:,:,:,0]
    X2 = np.asarray(X2)
    X2 = X2[:,:,:,0]
    Y = np.asarray(Y)
#     print("Pre-y shape", Y.shape)
    to_append_Y = 1*(np.sum(Y, axis=1) > 0) #Add the label for nothing wrong by checking if the other labels aren't zero and append
#     print("To_append y shape", to_append_Y.shape)
    to_append_Y = to_append_Y.reshape(to_append_Y.shape[0],1)
#     print("To_append y shape", to_append_Y.shape)
    Y = np.hstack((Y, to_append_Y))
#     print("X1:", X1.shape)
#     print("X1[0]:", X1[0].shape)
#     print("X2:", X2.shape)
#     print("Y:",Y.shape)
#     print("Y[0]:",Y[0].shape)
    d = [X1,X2,Y]
    return d

In [12]:
def data_gen(data,y,slice_no,model_no):
    d = []
    x = data[slice_no]
    x = np.asarray(x)
    y = np.asarray(y)
      #filtering all 0 slices and non-tumor slices
    if(x.any() != 0 and y.any() != 0):
        if(model_no == 0):
            X1 = []
            for i in range(16,224):##Will need to change to be 240 I believe for out images since this creates the image slices
                for j in range(16,224):
                    if(np.sum(x[i-16:i+17,j-16:j+17,:]) != 0): #Checks to makes sure data all nonzero
                        X1.append(x[i-16:i+17,j-16:j+17,:])
            Y1 = []
            for i in range(16,224):
                for j in range(16,224):
                    if(np.sum(x[i-16:i+17,j-16:j+17,:]) != 0):
                        Y1.append(y[slice_no,i,j]) 
            X1 = np.asarray(X1)
            X1 = X1[:,:,:,0]
            Y1 = np.asarray(Y1)
            d = [X1,Y1]
        elif(model_no == 1):
            d = model_gen(65,x,y,slice_no)
        elif(model_no == 2):
            d = model_gen(56,x,y,slice_no)
        elif(model_no == 3):
            d = model_gen(53,x,y,slice_no)  
        print("Returned")
    return d   

Model Definations

In [14]:
import keras
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Lambda,Concatenate
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Add
from keras.models import Model
from keras import regularizers
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.initializers import glorot_normal
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model

In [15]:
def two_path(X_input):
    # Local path Conv1
    X = Conv2D(64,(7,7),strides=(1,1),padding='valid')(X_input)
    # Batch-norm
    X = BatchNormalization()(X)
    X1 = Conv2D(64,(7,7),strides=(1,1),padding='valid')(X_input)
    X1 = BatchNormalization()(X1)
    # Max-out
    X = layers.Maximum()([X,X1])
    X = Conv2D(64,(4,4),strides=(1,1),padding='valid',activation='relu')(X)
  
    # Global path
    X2 = Conv2D(160,(13,13),strides=(1,1),padding='valid')(X_input)
    X2 = BatchNormalization()(X2)
    X21 = Conv2D(160,(13,13),strides=(1,1),padding='valid')(X_input)
    X21 = BatchNormalization()(X21)
    # Max-out
    X2 = layers.Maximum()([X2,X21])
  
    # Local path Conv2
    X3 = Conv2D(64,(3,3),strides=(1,1),padding='valid')(X)
    X3 = BatchNormalization()(X3)
    X31 =  Conv2D(64,(3,3),strides=(1,1),padding='valid')(X)
    X31 = BatchNormalization()(X31)
    X = layers.Maximum()([X3,X31])
    X = Conv2D(64,(2,2),strides=(1,1),padding='valid',activation='relu')(X)
  
    # Merging the two paths
    X = Concatenate()([X2,X])
    #X = Conv2D(5,(21,21),strides=(1,1))(X)
    #X = Activation('softmax')(X)
  
    #model = Model(inputs = X_input, outputs = X)
    return X

In [16]:
def input_cascade(input_shape1,input_shape2):
  
    X1_input = Input(input_shape1)
    # 1st two-path of cascade
    X1 = two_path(X1_input)
    X1 = Conv2D(5,(21,21),strides=(1,1),padding='valid',activation='relu')(X1)
    X1 = BatchNormalization()(X1)
  
    X2_input = Input(input_shape2)
    #Concatenating the output of 1st to input of 2nd
    X2_input1 = Concatenate()([X1,X2_input])
    #X2_input1 = Input(tensor = X2_input1)
    X2 = two_path(X2_input1)
    # Fully convolutional softmax classification
    X2 = Conv2D(5,(21,21),strides=(1,1),padding='valid')(X2)
    X2 = BatchNormalization()(X2)
    X2 = Activation('softmax')(X2)
    X2 = layers.Reshape((5,))(X2)
    model = Model(inputs=[X1_input,X2_input],outputs=X2)
    return model
  

In [17]:
def MFCcascade(input_shape1,input_shape2):
  
    # 1st two-path
    X1_input = Input(input_shape1)
    X1 = two_path(X1_input)
    X1 = Conv2D(5,(21,21),strides=(1,1),padding='valid',activation='relu')(X1)
    X1 = BatchNormalization()(X1)
    #X1 = MaxPooling2D((2,2))(X1)
  
    #2nd two-path 
    X2_input = Input(input_shape2)
    X2 = two_path(X2_input)
  
    # Concatenate before classification
    X2 = Concatenate()([X1,X2])
    X2 = Conv2D(5,(21,21),strides=(1,1),padding='valid',activation='relu')(X2)
    X2 = BatchNormalization()(X2)
    X2 = Activation('softmax')(X2)
    X2 = layers.Reshape((5,))(X2) #Added 
  
    model = Model(inputs=[X1_input,X2_input],outputs=X2)
    return model
  

In [18]:
def two_pathcnn(input_shape):
    X_input = Input(input_shape)
    X = Conv2D(64,(7,7),strides=(1,1),padding='valid')(X_input)
    X = BatchNormalization()(X)
    X1 = Conv2D(64,(7,7),strides=(1,1),padding='valid')(X_input)
    X1 = BatchNormalization()(X1)
    X = layers.Maximum()([X,X1])
    X = Conv2D(64,(4,4),strides=(1,1),padding='valid',activation='relu')(X)
  
    X2 = Conv2D(160,(13,13),strides=(1,1),padding='valid')(X_input)
    X2 = BatchNormalization()(X2)
    X21 = Conv2D(160,(13,13),strides=(1,1),padding='valid')(X_input)
    X21 = BatchNormalization()(X21)
    X2 = layers.Maximum()([X2,X21])
  
    X3 = Conv2D(64,(3,3),strides=(1,1),padding='valid')(X)
    X3 = BatchNormalization()(X3)
    X31 =  Conv2D(64,(3,3),strides=(1,1),padding='valid')(X)
    X31 = BatchNormalization()(X31)
    X = layers.Maximum()([X3,X31])
    X = Conv2D(64,(2,2),strides=(1,1),padding='valid',activation='relu')(X)
  
    X = Concatenate()([X2,X])
    X = Conv2D(5,(21,21),strides=(1,1),padding='valid')(X)
    X = Activation('softmax')(X)
    X = layers.Reshape((5,))(X)
  
    model = Model(inputs = X_input, outputs = X)
    return model 

In [19]:
m0 = two_pathcnn((33,33,1))
m0.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 33, 33, 4)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 27, 27, 64)   12608       ['input_1[0][0]']                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 27, 27, 64)   12608       ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 27, 27, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                   

In [20]:
m3 = MFCcascade((53,53,1),(33,33,1))
m3.summary()
# m3.save('trial_MFCcascade_acc.h5')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 53, 53, 4)]  0           []                               
                                                                                                  
 conv2d_9 (Conv2D)              (None, 47, 47, 64)   12608       ['input_2[0][0]']                
                                                                                                  
 conv2d_10 (Conv2D)             (None, 47, 47, 64)   12608       ['input_2[0][0]']                
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 47, 47, 64)  256         ['conv2d_9[0][0]']               
 rmalization)                                                                               

In [21]:
m1 = input_cascade((65,65,1),(33,33,1))
m1.summary()
# m1.save('trial_0001_input_cascasde_acc.h5')

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 65, 65, 1)]  0           []                               
                                                                                                  
 conv2d_27 (Conv2D)             (None, 59, 59, 64)   3200        ['input_4[0][0]']                
                                                                                                  
 conv2d_28 (Conv2D)             (None, 59, 59, 64)   3200        ['input_4[0][0]']                
                                                                                                  
 batch_normalization_20 (BatchN  (None, 59, 59, 64)  256         ['conv2d_27[0][0]']              
 ormalization)                                                                              

Training the architectures


In [22]:
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer

In [23]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [25]:
def specificity(y_true, y_pred):
    true_negatives = keras.sum(keras.round(keras.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = keras.sum(keras.round(keras.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + keras.epsilon())

In [26]:
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [27]:
def convert_yLabel_y5class(y_Label): #Converts the 4 segment output to 5, since the 5th label is supposed to indicate no label
    arr = list(y_Label)
    if np.sum(y_Label) == 0:
        arr.append(1)
    else:
        arr.append(0)
    return np.asarray(arr)

In [28]:
# Since sklearn 's compute_class_weights wasn't working, we found and used this file from here:
# https://gist.github.com/angeligareta/83d9024c5e72ac9ebc34c9f0b073c64c
def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.

  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}

  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed. 
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).

  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
        class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)
    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
        mlb = MultiLabelBinarizer()
        class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))

Training for the InputCascadeCNN model

In [29]:
#Note that the output here will look different since the random prints have been removed

with tf.device('/CPU:0'):
    fold1 = os.listdir(os.path.join(data_path, 'Training', 'Brains')) 
    fold2 = os.listdir(os.path.join(data_path, 'Training', 'Labels'))
    fold1.sort(key=str.lower) 
    fold2.sort(key=str.lower)

    arr = []
    Y_labels = []

    for B_Lpath, S_Lpath in zip(fold1, fold2):
        B_path = os.path.join(data_path, 'Training','Brains',B_Lpath)
        S_path = os.path.join(data_path, 'Training','Labels', S_Lpath)
        if "raw" not in B_path:
            brain_img = sitk.ReadImage(B_path)
            plt.show(brain_img)
            arr.append(sitk.GetArrayFromImage(brain_img))
            label_img = sitk.ReadImage(S_path)
            Y_label_array = sitk.GetArrayFromImage(label_img)
            Y_label = convert_y_2D_y_3D(Y_label_array)
            Y_labels.append(Y_label)
    arr = np.asarray(arr)
    Y_labels = np.asarray(Y_labels)
    data = arr
    data = data[..., np.newaxis]
    info = []
    
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.005)
    
    for i in range(data.shape[0]):
        d = data_gen(data,Y_labels,i,1) # Change this last parameter based on which model is being trained:
                                        # 0 : twopathCNN
                                        # 1 : InputCascadeCNN
                                        # 2 : LocalCascadeCNN (Model Not Implemented by Code's Developer)
                                        # 3 : MFCCascadeCNN
                                        #
        if(len(d) != 0):
            y = np.zeros((d[2].shape[0],5))
            for j in range(y.shape[0]):

                y[j] = d[2][j]
            X1 = d[0]
            X2 = d[1]
            y_ints = np.argmax(d[2], axis=1)
            print("Y_ints shape",y_ints.shape)
            print("d[2] shape", d[2].shape)
            print("NP.unique yints",  np.unique(y_ints))
            print("d[2][0]",d[2][0])
            class_weights = generate_class_weights(d[2], multi_class=False, one_hot_encoded=True)
            print("Class weights for this patch: ", class_weights)
            print(y.shape)
#             with tf.device('/GPU:0'): #If the GPU could have handled the sheer amount of data, of even batch size 1, 
                                        #this would be uncommented


            #Here, change m1 to the name of the model being trained
            m1.compile(optimizer = 'adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])#, specificity, f1_m])
            info.append(m1.fit([X1,X2],y,epochs=2,batch_size=8,class_weight= class_weights))

(200, 240, 240, 4)
(240, 240, 1)
Returned
Y_ints shape (4744,)
d[2] shape (4744, 5)
NP.unique yints [0 1 3]
d[2][0] [0 0 0 0 0]
Temp FIx {0: 8.704587155963303, 1: 7.53015873015873, 2: 1, 3: 10.093617021276597, 4: 2.8838905775075987}
{0: 0.34954317712938404, 1: 12.55026455026455, 2: 16.822695035460992}
(4744, 5)
Epoch 1/2


C:\Users\zooha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 3], y=[0 0 0 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


593/593 [==============================] - 596s 1s/step - loss: 1.5343 - accuracy: 0.1220
Epoch 2/2
593/593 [==============================] - 596s 1s/step - loss: 1.3345 - accuracy: 0.1109
(240, 240, 1)
Returned
Y_ints shape (4779,)
d[2] shape (4779, 5)
NP.unique yints [0 1 3]
d[2][0] [0 0 0 0 0]
Temp FIx {0: 8.932710280373831, 1: 7.834426229508197, 2: 1, 3: 10.061052631578947, 4: 2.95}
{0: 0.34918895221394125, 1: 13.057377049180328, 2: 16.768421052631577}
(4779, 5)
Epoch 1/2
598/598 [==============================] - 598s 998ms/step - loss: 1.3606 - accuracy: 0.0456
Epoch 2/2
598/598 [==============================] - 602s 1s/step - loss: 1.2829 - accuracy: 0.0506
(240, 240, 1)
Returned
Y_ints shape (5273,)
d[2] shape (5273, 5)
NP.unique yints [0 1 3]
d[2][0] [0 0 0 0 0]
Temp FIx {0: 24.525581395348837, 1: 13.1825, 2: 1, 3: 10.872164948453609, 4: 4.793636363636364}
{0: 0.34491104133961276, 1: 21.970833333333335, 2: 18.120274914089347}
(5273, 5)
Epoch 1/2
660/660 [====================

729/729 [==============================] - 726s 995ms/step - loss: 1.0061 - accuracy: 0.0954
Epoch 2/2
729/729 [==============================] - 724s 993ms/step - loss: 1.0049 - accuracy: 0.0966
(240, 240, 1)
Returned
Y_ints shape (5869,)
d[2] shape (5869, 5)
NP.unique yints [0 1 3]
d[2][0] [0 0 0 0 0]
Temp FIx {0: 8.505797101449275, 1: 3.691194968553459, 2: 1, 3: 2.949246231155779, 4: 1.374473067915691}
{0: 0.3796493951743321, 1: 6.151991614255765, 2: 4.915410385259632}
(5869, 5)
Epoch 1/2
734/734 [==============================] - 730s 993ms/step - loss: 1.0023 - accuracy: 0.0901
Epoch 2/2
734/734 [==============================] - 732s 998ms/step - loss: 0.9729 - accuracy: 0.1036
(240, 240, 1)
Returned
Y_ints shape (5895,)
d[2] shape (5895, 5)
NP.unique yints [0 1 3]
d[2][0] [0 0 0 0 0]
Temp FIx {0: 11.228571428571428, 1: 3.5194029850746267, 2: 1, 3: 2.977272727272727, 4: 1.4102870813397128}
{0: 0.3805189775367932, 1: 5.865671641791045, 2: 4.962121212121212}
(5895, 5)
Epoch 1/2
737

C:\Users\zooha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0], y=[0 0 0 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


654/654 [==============================] - 647s 987ms/step - loss: 0.0000e+00 - accuracy: 0.0272
Epoch 2/2
654/654 [==============================] - 645s 986ms/step - loss: 0.0000e+00 - accuracy: 0.0258
(240, 240, 1)
Returned
Y_ints shape (5261,)
d[2] shape (5261, 5)
NP.unique yints [0]
d[2][0] [0 0 0 0 0]
Temp FIx {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: 1.0}
(5261, 5)
Epoch 1/2
658/658 [==============================] - 655s 994ms/step - loss: 0.0000e+00 - accuracy: 0.0283
Epoch 2/2
658/658 [==============================] - 654s 994ms/step - loss: 0.0000e+00 - accuracy: 0.0293
(240, 240, 1)
Returned
Y_ints shape (5307,)
d[2] shape (5307, 5)
NP.unique yints [0]
d[2][0] [0 0 0 0 0]
Temp FIx {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: 1.0}
(5307, 5)
Epoch 1/2
664/664 [==============================] - 665s 1000ms/step - loss: 0.0000e+00 - accuracy: 0.0256
Epoch 2/2
664/664 [==============================] - 665s 1s/step - loss: 0.0000e+00 - accuracy: 0.0307
(240, 240, 1)
Returned
Y_ints shape (5380,

C:\Users\zooha\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1], y=[0 0 0 ... 0 0 0] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


673/673 [==============================] - 678s 1s/step - loss: 0.3481 - accuracy: 0.0292
Epoch 2/2
673/673 [==============================] - 673s 1s/step - loss: 0.2919 - accuracy: 0.0212
(240, 240, 1)
Returned
Y_ints shape (5423,)
d[2] shape (5423, 5)
NP.unique yints [0 1 3]
d[2][0] [0 0 0 0 0]
Temp FIx {0: 1, 1: 542.3, 2: 1, 3: 542.3, 4: 271.15}
{0: 0.33357938118964137, 1: 903.8333333333334, 2: 903.8333333333334}
(5423, 5)
Epoch 1/2
678/678 [==============================] - 681s 1s/step - loss: 0.9834 - accuracy: 0.0227
Epoch 2/2
678/678 [==============================] - 680s 1s/step - loss: 0.6957 - accuracy: 0.0343
(240, 240, 1)
Returned
Y_ints shape (5475,)
d[2] shape (5475, 5)
NP.unique yints [0 1 3]
d[2][0] [0 0 0 0 0]
Temp FIx {0: 1095.0, 1: 219.0, 2: 1, 3: 57.63157894736842, 4: 43.8}
{0: 0.33480095395340304, 1: 365.0, 2: 96.05263157894737}
(5475, 5)
Epoch 1/2
685/685 [==============================] - 688s 1s/step - loss: 2.0953 - accuracy: 0.0336
Epoch 2/2
685/685 [======

KeyboardInterrupt: 

In [ ]:
label_img = sitk.ReadImage(os.path.join(data_path, "Training/Labels/seg_0.mhd"))
Y_label_array = sitk.GetArrayFromImage(label_img)
Y_label = convert_y_2D_y_3D(Y_label_array)
plt.imshow(Y_label_array)

Testing

In [1]:
fold1 = os.listdir(os.path.join(data_path, 'Testing', 'Brains')) 
fold2 = os.listdir(os.path.join(data_path, 'Testing', 'Labels'))
fold1.sort(key=str.lower) 
fold2.sort(key=str.lower)

arr = []
Y_labels = []

for B_Lpath, S_Lpath in zip(fold1, fold2):
    B_path = os.path.join(data_path, 'Testing','Brains',B_Lpath)
    S_path = os.path.join(data_path, 'Testing','Labels', S_Lpath)
    if "raw" not in B_path:
        brain_img = sitk.ReadImage(B_path)
        plt.show(brain_img)
        arr.append(sitk.GetArrayFromImage(brain_img))
        label_img = sitk.ReadImage(S_path)
        Y_label_array = sitk.GetArrayFromImage(label_img)
        Y_label = convert_y_2D_y_3D(Y_label_array)
        Y_labels.append(Y_label)
arr = np.asarray(arr)
Y_labels = np.asarray(Y_labels)
data = arr
data = data[..., np.newaxis]
info = []

d = data_gen(data,Y_labels,22,1) #The 3rd argument is the Image we want to predict based on its patches
if(len(d) != 0):
    y = np.zeros((d[2].shape[0],5))
    for j in range(y.shape[0]):
        y[j] = d[2][j]
    X1 = d[0]
    X2 = d[1]
    pred = m1.predict([X1,X2],batch_size = 256) 
    pred = np.around(pred)
    #print(pred.shape)
    pred1 = np.argmax(pred.reshape(y.shape[0],5)[:,1:4],axis = 1)
    y2 = np.argmax(y.reshape(y.shape[0],5)[:,0:5],axis = 1)
    f1 = metrics.f1_score(y2,pred1,average='micro')
    print(f1)

NameError: name 'os' is not defined

In [ ]:
pred = model.predict([X1,X2],batch_size = 256)

In [ ]:
pred = np.around(pred)
print(pred.shape)
pred1 = np.argmax(pred.reshape(y.shape[0],5)[:,1:4],axis = 1)
y2 = np.argmax(y.reshape(y.shape[0],5)[:,1:4],axis = 1)

In [ ]:
f1 = metrics.f1_score(y2,pred1,average='micro')
f1

In [ ]:
pred2 = m1.predict([X1,X2],batch_size = 256)

In [ ]:
pred2 = np.around(pred2)
pred3 = np.argmax(pred2.reshape(y.shape[0],5)[:,1:4],axis = 1)
y2 = np.argmax(y.reshape(y.shape[0],5)[:,1:4],axis = 1)

In [ ]:
f1 = metrics.f1_score(y2,pred3,average='micro')
f1